In [ ]:
#SWN20140227 Table: 0
#remaining bug: bcf in the first column...

In [14]:
%load_ext autoreload
%autoreload 2
import pickle
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from zlu_functions import del_subset_col, set_col_headers, \
                          row_header_to_col,remove_rows_for_total_value, \
                          dfs_need_transpose, remove_cols_for_fair_value, \
                          collar_fill_empty,remove_empty_rows_and_cols, \
                          df_column_uniquify, verticalSplit_instrument, \
                          verticalSplit_ZLU
from nel_functions import delete_uninformative,delete_empty,delete_duplicated, \
                          delete_subset,get_vertically_stacked_subtables, \
                          vertically_stacked_subtable_cleaner
from helper import volumeCleaner, dateCleaner, priceCleaner,trimDictonary,verticalSplit, \
                   instrumentCleaner, productCleaner, filterBusiness,columnSorter, \
                   verticalSplit_test
#pickle_in=open('../SEC_Data/Hedge/hedge_dfs_all_new.pickle','rb')

pickle_in=open('hedge_dfs_sub_new.pickle', 'rb')
df_dic=pickle.load(pickle_in)

delete_empty(df_dic)
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
del_dfs_dic_0 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
del_dfs_dic_1 = delete_subset(df_dic)

ticker_list = []

for k in df_dic.keys():
    if k[:3] not in ticker_list:
        print(k[:3])
        ticker_list.append(k[:3])


df_dic = filterBusiness(df_dic, 'CLR')
# df_dic={'CLR20100226':df_dic['CLR20100226']}
#del df_dic['PKD20090302'] #false positive
with open('output_before_spliting.csv','w') as f:
    pass
#preprocessing for all dfs
for k in df_dic.keys():
    for i in range(len(df_dic[k])):

        df=df_dic[k][i]
        df=set_col_headers(df) #set headers
        df=del_subset_col(df) #delete redundant cols
        df=remove_rows_for_total_value(df) #if any row for total value; save it
        df=remove_cols_for_fair_value(df)
        df_dic[k][i]=df
        with open('output_before_spliting.csv','a') as f:
            df=df_dic[k][i]
            f.writelines(k+'\n'+'table:'+str(i))
            try: df.to_csv(f,encoding='cp1252')
            except: df.to_csv(f,encoding='utf-8')

#         print(k,'Original Table:',i,'-'*10)
#         display(HTML(df.to_html()))

#Vertical Split
# for k in df_dic.keys(): #apply Sam's spliting function
    #print(k)
#     if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
#     elif k=='BRY20110301':
#         split_dfs = []
#         for table in df_dic[k]:
#             df1 = table.iloc[:, 0:3]
#             df2 = table.iloc[:, 3:]
#             split_dfs.append(df1)
#             split_dfs.append(df2)
#             df_dic[k]=split_dfs
#     else: df_dic[k] = verticalSplit(df_dic[k])
#     df_dic[k]=verticalSplit_instrument(df_dic[k])
#     if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
#     else: df_dic[k] = verticalSplit_ZLU(df_dic[k])
 

for k in df_dic.keys():
    #print(k)
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df=remove_rows_for_total_value(df) #if any row for total value; save it
            df=remove_cols_for_fair_value(df)
            df=row_header_to_col(df)
            df=dfs_need_transpose(df)
            df=collar_fill_empty(df)
            df=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
#             print(k,'Table:',i,'-'*10,'After preprocessing')
#             display(HTML(df.to_html()))


split_dictonary = {'swap_split' : ['swap'],
                   'collar_split': ['collar', 'floor', 'ceiling']}


#Vertical Split
for k in df_dic.keys(): #apply Sam's spliting function
    print(k)
    df_dic[k] = verticalSplit_test(df_dic[k], dictonary = split_dictonary, verbose = True)
#     if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
#     else: df_dic[k] = verticalSplit_ZLU(df_dic[k])
#     df_dic[k]=verticalSplit_instrument(df_dic[k])
#     if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
#     else: df_dic[k] = verticalSplit_ZLU(df_dic[k])    
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df=remove_rows_for_total_value(df) #if any row for total value; save it
            df=remove_cols_for_fair_value(df)
            df=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
           
#Horizontally Split
del_dfs_dic3=get_vertically_stacked_subtables(df_dic)
vertically_stacked_subtable_cleaner(df_dic,del_dfs_dic3,exclusion=True, excluded_ticker=('MXC', 'OXY', 'HOC'))

delete_empty(df_dic)
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
del_dfs_dic_2 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
del_dfs_dic_3 = delete_subset(df_dic)

for k in df_dic.keys(): #apply Sam's spliting function
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df_dic[k][i]=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
            
with open('output_before_sam.csv','w') as f:
    pass
with open('output_after_sam.csv','w') as f:
    pass
for k in df_dic.keys():
    for i in range(len(df_dic[k])):
        with open('output_before_sam.csv','a') as f:
            df=df_dic[k][i]
            f.writelines(k+'\n'+'table:'+str(i))
            try: df.to_csv(f,encoding='cp1252')
            except: df.to_csv(f,encoding='utf-8')

date_keys = []               
date_stop = []

price_keys = ['weighted average floor price per mcf','weighted average price per mcf',
              'weighted average price per bbl', 'weighted average floor price per bbl',
              'weighted,average,price to be,swapped,($/mmbtu),,', 'weighted,average,floor,price,($/mmbtu),,',
              'weighted,average,ceiling,price,($/mmbtu),,', 'weighted,average,basis,differential,($/mmbtu),,',
              'weighted average price per mmbtu', 'weighted average price', 'weighted average floor price',
              'weighted average ceiling price', 'weighted average basis differential','swapsweightedaverage','call','floor','ceiling']

price_stop = []

volume_keys = ['mbbls','mmmbtu','mmbtu','bbls']
volume_stop = ['natural gas (bcf)']               

#set verbose to True to turn on the diagonstic output for Sam's functions
verbose = False
key_override = True
                
for k in df_dic.keys():
    print(k)
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
#         display(HTML(df.to_html()))
        if df.shape[0]>0: #apply Sam's functions to non-empty dfs
            print(k,'Table:',i,'-'*10,'Before Sam function')
            col_lst=[re.sub("[,]+","",col) if 'product_type' in col or 'instrument_type' in col else col for col in df.columns]
            df.columns=col_lst
            display(HTML(df.to_html()))
            df = instrumentCleaner(df, key_words = "", stop_words = "", verbose = verbose)
            df = productCleaner(df, key_words = "", stop_words = "", verbose = verbose)
            df = priceCleaner(df, key_words = price_keys, stop_words = price_stop,\
                                  verbose = verbose, key_override = key_override)
            df = volumeCleaner(df, key_words = volume_keys, stop_words = volume_stop,\
                                   verbose = verbose, key_override = key_override)
            df = dateCleaner(df, value_ids = ["20\d{2}"])
            #ZLU's change begins
            col_lst=[col for col in df.columns if 'vol' in col]
            for col in col_lst:
                df.loc[:,col]=df.loc[:,col].str.replace('[,\(\)]',"")
    #         display(HTML(df.to_html()))
    
    # build into functions volumeCleaner, row delete can stay here
            col_lst=['vol' in col for col in df.columns]
            row_lst=list(range(df.shape[0]))
            for row_idx in range(df.shape[0]):
                if not any(df.iloc[row_idx,col_lst].str.contains('\d')):
                    row_lst.remove(row_idx)
            df=df.iloc[row_lst]
            if all(df.loc[:,'vol_1_type']=='volume'): df.loc[:,'vol_1_type']=df.loc[:,'product_type'].str.extract('\(([^\)]+)\)')
            #ZLU's change ends
            
            print(k+' ,table:'+str(i)+'-----After Sam functions----')
            display(HTML(df.to_html()))
            print('~'*50)
        else: df=pd.DataFrame()
        df_dic[k][i]=df

del_dfs_dic_5 = delete_uninformative(df_dic)
print('1',len(df_dic))
delete_empty(df_dic)
print('2',len(df_dic))
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
print('3',len(df_dic))
del_dfs_dic_2 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
print('4',len(df_dic))
del_dfs_dic_3 = delete_subset(df_dic)
print('5',len(df_dic))

print(len(df_dic.keys()))

big_df = pd.DataFrame()

for k in df_dic.keys():
    doc_id = k
    processed_dfs = df_dic[k]
    for df in processed_dfs:
        unprocessed_cols = columnSorter(df)
        print(unprocessed_cols)
        df.drop(unprocessed_cols, axis = 1, inplace = True)
             #zlu change starts
        col_lst=['price' in col for col in df.columns]
        row_lst=list(range(df.shape[0]))
        df['red_flag']=''
        for row_idx in range(df.shape[0]):
            if not any(df.iloc[row_idx,col_lst].str.contains('\d')):
                df.iloc[row_idx,-1]=1
        #zlu change ends

#         display(HTML(df.to_html()))
        length = df.shape[0]
        doc_id_col = np.array([doc_id]*length)
        df.insert(0, 'doc_id', doc_id_col)
        big_df = pd.concat([big_df, df], axis = 0, join = 'outer', sort = False)

print(big_df.shape)

if any(big_df['red_flag'] != ''):
    print("Check the red_flag column for missing values at")
    #print('\n\n', big_df.iloc[big_df['red_flag'] != '', ].index)

try: big_df.to_csv('CLR_clean.csv', encoding='cp1252')
except: big_df.to_csv('CLR_clean.csv', encoding = 'utf-8')
# for k in df_dic.keys():
#     for i in range(len(df_dic[k])):
#         with open('APA_clean.csv','a') as f:
#             df=df_dic[k][i]
#             f.writelines(k+'\n'+'table:'+str(i))
#             try: df.to_csv(f,encoding='cp1252')
#             except: df.to_csv(f,encoding='utf-8')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
APA
APC
CLR
CPE
DVN
ECA
HES
MRO
MUR
NBL
OXY
PXD
RRC
SWN
UNT
CLR20100226
Splitting on  swap_split
Split  0 columns:
 ['period and type of contract,period and type of contract,period and type of contract', 'volumeinmbbls,volumeinmbbls,volumeinmbbls', 'swapsweightedaverage,swapsweightedaverage,swapsweightedaverage', 'additional']
Split  0 pure columns:
 ['swapsweightedaverage,swapsweightedaverage,swapsweightedaverage']


,"period and type of contract,period and type of contract,period and type of contract","volumeinmbbls,volumeinmbbls,volumeinmbbls","swapsweightedaverage,swapsweightedaverage,swapsweightedaverage",additional
4,Swaps,343,$84.00,january 2010 march 2010
5,Collars,148,NaN,january 2010 march 2010
7,Swaps,410,84.22,april 2010 june 2010
8,Collars,228,NaN,april 2010 june 2010
10,Swaps,414,84.22,july 2010 september 2010
11,Collars,598,NaN,july 2010 september 2010
13,Swaps,414,84.22,october 2010 december 2010
14,Collars,598,NaN,october 2010 december 2010


,"period and type of contract,period and type of contract,period and type of contract","volumeinmbbls,volumeinmbbls,volumeinmbbls","swapsweightedaverage,swapsweightedaverage,swapsweightedaverage",additional
4,Swaps,343,$84.00,january 2010 march 2010
7,Swaps,410,84.22,april 2010 june 2010
10,Swaps,414,84.22,july 2010 september 2010
13,Swaps,414,84.22,october 2010 december 2010


Splitting on  collar_split
Split  1 columns:
 ['period and type of contract,period and type of contract,period and type of contract', 'volumeinmbbls,volumeinmbbls,volumeinmbbls', 'collars,floors,range', 'collars,floors,weightedaverage', 'collars,ceilings,range', 'collars,ceilings,weightedaverage', 'additional']
Split  1 pure columns:
 ['collars,floors,range', 'collars,floors,weightedaverage', 'collars,ceilings,range', 'collars,ceilings,weightedaverage']


,"period and type of contract,period and type of contract,period and type of contract","volumeinmbbls,volumeinmbbls,volumeinmbbls","collars,floors,range","collars,floors,weightedaverage","collars,ceilings,range","collars,ceilings,weightedaverage",additional
4,Swaps,343,NaN,NaN,NaN,NaN,january 2010 march 2010
5,Collars,148,75.00,75.00,96.40,96.40,january 2010 march 2010
7,Swaps,410,NaN,NaN,NaN,NaN,april 2010 june 2010
8,Collars,228,75.00,75.00,96.40,96.40,april 2010 june 2010
10,Swaps,414,NaN,NaN,NaN,NaN,july 2010 september 2010
11,Collars,598,75.00,75.00,94.50|96.40,95.23,july 2010 september 2010
13,Swaps,414,NaN,NaN,NaN,NaN,october 2010 december 2010
14,Collars,598,75.00,75.00,94.50|96.40,95.23,october 2010 december 2010


...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
CLR20170222 Table: 2 ---------- Before Sam function


,",,,period and type of contract",",,,mmbtus",",floors,,range",",floors,weighted average price,weighted average price",",ceilings,,range",",ceilings,weighted average price,weighted average price",additional
6,Collars - Henry Hub,65700000,$2.40|$3.00,$2.47,$2.92|$3.88,$3.08,january 2017 - december 2017


instrument_type column already exists
CLR20170222 ,table:2-----After Sam functions----


,",,,period and type of contract",vol_1,vol_1_type,price_1,price_1_type,price_2,price_2_type,price_3,price_3_type,price_4,price_4_type,maturity,instrument_type,product_type
6,Collars - Henry Hub,65700000,mmbtu,$2.40|$3.00,floor,$2.47,floor,$2.92|$3.88,ceiling,$3.08,ceiling,20172017,collar,natural gas


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 8
2 8
3 8
4 8
5 8
8
['period and type of contract,period and type of contract,period and type of contract']
['period and type of contract,period and type of contract,period and type of contract']
['period and type of contract']
[',,period and type of contract']
[',,period and type of contract']
['period and type of contract']
['period and type of contract']
[',,period and type of contract']
[',,period and type of contract']
['period and type of contract']
[',,,period and type of contract']
[',,,period and type of contract']
['period and type of contract']
['period and type of contract']
['period and type of contract']
['crude oil west texas intermediate,,period and type of contract,period and type of contract']
['crude oil west texas intermediate,,period and type of contract,period and type of contract']
['natural gas henry hub,period and type of contract']
['period and type of contract']
['period and type of contract']
['crude oil n

In [3]:
big_df[big_df['red_flag'] == 1]

,doc_id,vol_1,vol_1_type,price_1,price_1_type,maturity,instrument_type,product_type,red_flag,price_2,price_2_type,price_3,price_3_type,price_4,price_4_type,price_5,price_5_type
5,CLR20100226,148,mbbls,,swapsweightedaverage,20102010,collar,oil,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CLR20100226,228,mbbls,,swapsweightedaverage,20102010,collar,oil,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,CLR20100226,598,mbbls,,swapsweightedaverage,20102010,collar,oil,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,CLR20100226,598,mbbls,,swapsweightedaverage,20102010,collar,oil,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CLR20100226,343,mbbls,,floor,20102010,swap,oil,1,,floor,,ceiling,,ceiling,NaN,NaN
7,CLR20100226,410,mbbls,,floor,20102010,swap,oil,1,,floor,,ceiling,,ceiling,NaN,NaN
10,CLR20100226,414,mbbls,,floor,20102010,swap,oil,1,,floor,,ceiling,,ceiling,NaN,NaN
13,CLR20100226,414,mbbls,,floor,20102010,swap,oil,1,,floor,,ceiling,,ceiling,NaN,NaN
5,CLR20100226,453,mbbls,,swapsweightedaverage,20102010,collar,oil,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CLR20100226,644,mbbls,,swapsweightedaverage,20102010,collar,oil,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
re.search('\d', '-')